##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

注：您现在可以在 Jupyter 风格的笔记本中运行此示例而无需进行设置！只需点击“在 Google Colab 中运行”

<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td><a target="_blank" href="https://tensorflow.google.cn/tfx/tutorials/model_analysis/tfma_basic"><img src="https://tensorflow.google.cn/images/tf_logo_32px.png">在 TensorFlow.org上查看</a></td>
<td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/zh-cn/tfx/tutorials/model_analysis/tfma_basic.ipynb"><img src="https://tensorflow.google.cn/images/colab_logo_32px.png">在 Google Colab 运行</a>
</td>
<td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/zh-cn/tfx/tutorials/model_analysis/tfma_basic.ipynb"><img width="32px" src="https://tensorflow.google.cn/images/GitHub-Mark-32px.png">在 GitHub 上查看源代码</a>
</td>
<td><a target="_blank" href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/zh-cn/tfx/tutorials/model_analysis/tfma_basic.ipynb"><img width="32px" src="https://tensorflow.google.cn/images/download_logo_32px.png">下载笔记本</a></td>
</table></div>

# TensorFlow Model Analysis

***TensorFlow Extended (TFX) 关键组件示例***

[TensorFlow Model Analysis (TFMA)](https://tensorflow.google.cn/tfx/guide/tfma) 是用于对不同数据切片执行模型评估的库。TFMA 使用 [Apache Beam](https://beam.apache.org/documentation/programming-guide/) 以分布式的方式对大量数据执行计算。

此示例 Colab 笔记本将演示如何使用 TFMA 对数据集的特点和模型的性能进行调查和可视化。我们将使用之前训练的模型，现在您可以测试结果了！我们训练的模型本来用于[芝加哥出租车示例](https://github.com/tensorflow/tfx/tree/master/tfx/examples/chicago_taxi_pipeline)，该示例使用芝加哥市发布的 [Taxi Trips 数据集](https://data.cityofchicago.org/Transportation/Taxi-Trips/wrvz-psew)。您可以在 [BigQuery 界面](https://bigquery.cloud.google.com/dataset/bigquery-public-data:chicago_taxi_trips)中探索完整的数据集。

作为建模者和开发者，请思考如何使用这些数据以及模型预测的潜在好处和危害。此类模型可能会加剧社会偏见和差距。某个特征是与您要解决的问题相关，还是会引入偏见？有关详情，请阅读 <a target="_blank" href="https://developers.google.com/machine-learning/fairness-overview/">ML 公平性</a>。

注：要了解 TFMA 以及它如何与 Apache Beam 配合使用，您需要对 Apache Beam 有所了解。最好是从 <a target="_blank" href="https://beam.apache.org/documentation/programming-guide/">Beam 编程指南</a>开始着手。

数据集中的各列为：

<table>
<tr>
<td>pickup_community_area</td>
<td>fare</td>
<td>trip_start_month</td>
</tr>
<tr>
<td>trip_start_hour</td>
<td>trip_start_day</td>
<td>trip_start_timestamp</td>
</tr>
<tr>
<td>pickup_latitude</td>
<td>pickup_longitude</td>
<td>dropoff_latitude</td>
</tr>
<tr>
<td>dropoff_longitude</td>
<td>trip_miles</td>
<td>pickup_census_tract</td>
</tr>
<tr>
<td>dropoff_census_tract</td>
<td>payment_type</td>
<td>company</td>
</tr>
<tr>
<td>trip_seconds</td>
<td>dropoff_community_area</td>
<td>tips</td>
</tr>
</table>

## 安装 Jupyter 扩展程序

注：如果在本地 Jupyter 笔记本中运行，则必须在运行 Jupyter 之前在环境中安装以下 Jupyter 扩展程序。

```bash
jupyter nbextension enable --py widgetsnbextension --sys-prefix
jupyter nbextension install --py --symlink tensorflow_model_analysis --sys-prefix
jupyter nbextension enable --py tensorflow_model_analysis --sys-prefix
```

## 安装 TensorFlow Model Analysis (TFMA)

这将拉取所有依赖项，并需要花点时间。


In [ ]:
# Upgrade pip to the latest, and install TFMA.
!pip install -U pip
!pip install tensorflow-model-analysis

**现在，您必须在运行下面的单元之前重新启动运行时。**

In [ ]:
# This setup was tested with TF 2.10 and TFMA 0.41 (using colab), but it should
# also work with the latest release.
import sys

# Confirm that we're using Python 3
assert sys.version_info.major==3, 'This notebook must be run using Python 3.'

import tensorflow as tf
print('TF version: {}'.format(tf.__version__))
import apache_beam as beam
print('Beam version: {}'.format(beam.__version__))
import tensorflow_model_analysis as tfma
print('TFMA version: {}'.format(tfma.__version__))

**注：在继续之前，上面的输出应没有错误。如果仍然看到错误，请重新运行安装。另外，在继续下一步之前，请确保重新启动运行时/内核。**

## 加载文件

我们将下载包含全部所需内容的 TAR 文件。内容包括：

- 训练数据集和评估数据集
- 数据架构
- 训练和应用已保存的模型（Keras 和 Estimator），以及评估已保存的模型 (Estimator)。

In [ ]:
# Download the tar file from GCP and extract it
import io, os, tempfile
TAR_NAME = 'saved_models-2.2'
BASE_DIR = tempfile.mkdtemp()
DATA_DIR = os.path.join(BASE_DIR, TAR_NAME, 'data')
MODELS_DIR = os.path.join(BASE_DIR, TAR_NAME, 'models')
SCHEMA = os.path.join(BASE_DIR, TAR_NAME, 'schema.pbtxt')
OUTPUT_DIR = os.path.join(BASE_DIR, 'output')

!curl -O https://storage.googleapis.com/artifacts.tfx-oss-public.appspot.com/datasets/{TAR_NAME}.tar
!tar xf {TAR_NAME}.tar
!mv {TAR_NAME} {BASE_DIR}
!rm {TAR_NAME}.tar

print("Here's what we downloaded:")
!ls -R {BASE_DIR}

## 解析架构

我们下载的内容包括由 [TensorFlow Data Validation](https://tensorflow.google.cn/tfx/data_validation/get_started/) 创建的数据架构。现在我们来解析架构，以便将其用于 TFMA。

In [ ]:
import tensorflow as tf
from google.protobuf import text_format
from tensorflow.python.lib.io import file_io
from tensorflow_metadata.proto.v0 import schema_pb2
from tensorflow.core.example import example_pb2

schema = schema_pb2.Schema()
contents = file_io.read_file_to_string(SCHEMA)
schema = text_format.Parse(contents, schema)

## 使用架构创建 TFRecord

我们需要授予 TFMA 访问数据集的权限，因此我们创建一个 TFRecord 文件。可以使用架构进行创建，因为它会为每个特征提供正确的类型。

In [ ]:
import csv

datafile = os.path.join(DATA_DIR, 'eval', 'data.csv')
reader = csv.DictReader(open(datafile, 'r'))
examples = []
for line in reader:
  example = example_pb2.Example()
  for feature in schema.feature:
    key = feature.name
    if feature.type == schema_pb2.FLOAT:
      example.features.feature[key].float_list.value[:] = (
          [float(line[key])] if len(line[key]) > 0 else [])
    elif feature.type == schema_pb2.INT:
      example.features.feature[key].int64_list.value[:] = (
          [int(line[key])] if len(line[key]) > 0 else [])
    elif feature.type == schema_pb2.BYTES:
      example.features.feature[key].bytes_list.value[:] = (
          [line[key].encode('utf8')] if len(line[key]) > 0 else [])
  # Add a new column 'big_tipper' that indicates if tips was > 20% of the fare. 
  # TODO(b/157064428): Remove after label transformation is supported for Keras.
  big_tipper = float(line['tips']) > float(line['fare']) * 0.2
  example.features.feature['big_tipper'].float_list.value[:] = [big_tipper]
  examples.append(example)

tfrecord_file = os.path.join(BASE_DIR, 'train_data.rio')
with tf.io.TFRecordWriter(tfrecord_file) as writer:
  for example in examples:
    writer.write(example.SerializeToString())

!ls {tfrecord_file}

## 设置并运行 TFMA

TFMA 支持许多不同的模型类型，包括 TF Keras 模型、基于通用 TF2 签名 API 的模型，以及基于 TF Estimator 的模型。[get_started](https://tensorflow.google.cn/tfx/model_analysis/get_started) 指南中有受支持的模型类型的完整列表及任何局限。在本示例中，我们将展示如何配置基于 Keras 的模型和 保存为 [`EvalSavedModel`](https://tensorflow.google.cn/tfx/model_analysis/eval_saved_model) 的基于 Estimator 的模型。请参阅 [FAQ](https://tensorflow.google.cn/tfx/model_analysis/faq) 了解其他配置的示例。

TFMA 支持计算训练时使用的指标（即内置指标）和模型作为 TFMA 配置设置的一部分保存后定义的指标。对于我们的 Keras [设置](https://tensorflow.google.cn/tfx/model_analysis/setup)，我们将演示如何将指标和绘图手动添加为配置的一部分（有关受支持的指标和绘图的信息，请参阅[指标](https://tensorflow.google.cn/tfx/model_analysis/metrics)指南）。对于 Estimator 设置，我们将使用与模型一起保存的内置指标。我们的设置还包括许多切片规范，这些规范将在以下部分详细讨论。

创建 [`tfma.EvalConfig`](https://tensorflow.google.cn/tfx/model_analysis/api_docs/python/tfma/EvalConfig) 和 [`tfma.EvalSharedModel`](https://tensorflow.google.cn/tfx/model_analysis/api_docs/python/tfma/EvalSharedModel) 后，我们可以使用 [`tfma.run_model_analysis`](https://tensorflow.google.cn/tfx/model_analysis/api_docs/python/tfma/run_model_analysis) 运行 TFMA。这将创建一个 [`tfma.EvalResult`](https://tensorflow.google.cn/tfx/model_analysis/api_docs/python/tfma/EvalResult)，稍后我们可以用它来呈现指标和绘图。

### Keras

In [ ]:
import tensorflow_model_analysis as tfma

# Setup tfma.EvalConfig settings
keras_eval_config = text_format.Parse("""
  ## Model information
  model_specs {
    # For keras (and serving models) we need to add a `label_key`.
    label_key: "big_tipper"
  }

  ## Post training metric information. These will be merged with any built-in
  ## metrics from training.
  metrics_specs {
    metrics { class_name: "ExampleCount" }
    metrics { class_name: "AUC" }
    metrics { class_name: "Precision" }
    metrics { class_name: "Recall" }
    metrics { class_name: "MeanPrediction" }
    metrics { class_name: "Calibration" }
    metrics { class_name: "CalibrationPlot" }
    metrics { class_name: "ConfusionMatrixPlot" }
    # ... add additional metrics and plots ...
  }

  ## Slicing information
  slicing_specs {}  # overall slice
  slicing_specs {
    feature_keys: ["trip_start_hour"]
  }
  slicing_specs {
    feature_keys: ["trip_start_day"]
  }
  slicing_specs {
    feature_values: {
      key: "trip_start_month"
      value: "1"
    }
  }
""", tfma.EvalConfig())

# Create a tfma.EvalSharedModel that points at our keras model.
keras_model_path = os.path.join(MODELS_DIR, 'keras', '2')
keras_eval_shared_model = tfma.default_eval_shared_model(
    eval_saved_model_path=keras_model_path,
    eval_config=keras_eval_config)

keras_output_path = os.path.join(OUTPUT_DIR, 'keras')

# Run TFMA
keras_eval_result = tfma.run_model_analysis(
    eval_shared_model=keras_eval_shared_model,
    eval_config=keras_eval_config,
    data_location=tfrecord_file,
    output_path=keras_output_path)

### Estimator

In [ ]:
import tensorflow_model_analysis as tfma

# Setup tfma.EvalConfig settings
estimator_eval_config = text_format.Parse("""
  ## Model information
  model_specs {
    # To use EvalSavedModel set `signature_name` to "eval".
    signature_name: "eval"
  }

  ## Post training metric information. These will be merged with any built-in
  ## metrics from training.
  metrics_specs {
    metrics { class_name: "ConfusionMatrixPlot" }
    # ... add additional metrics and plots ...
  }

  ## Slicing information
  slicing_specs {}  # overall slice
  slicing_specs {
    feature_keys: ["trip_start_hour"]
  }
  slicing_specs {
    feature_keys: ["trip_start_day"]
  }
  slicing_specs {
    feature_values: {
      key: "trip_start_month"
      value: "1"
    }
  }
""", tfma.EvalConfig())

# Create a tfma.EvalSharedModel that points at our eval saved model.
estimator_base_model_path = os.path.join(
    MODELS_DIR, 'estimator', 'eval_model_dir')
estimator_model_path = os.path.join(
    estimator_base_model_path, os.listdir(estimator_base_model_path)[0])
estimator_eval_shared_model = tfma.default_eval_shared_model(
    eval_saved_model_path=estimator_model_path,
    eval_config=estimator_eval_config)

estimator_output_path = os.path.join(OUTPUT_DIR, 'estimator')

# Run TFMA
estimator_eval_result = tfma.run_model_analysis(
    eval_shared_model=estimator_eval_shared_model,
    eval_config=estimator_eval_config,
    data_location=tfrecord_file,
    output_path=estimator_output_path)

## 呈现指标和绘图

现在，我们已经运行了评估，让我们看一下使用 TFMA 的可视化效果。对于以下示例，我们将呈现在 Keras 模型上运行评估的结果。要查看基于 Estimator 的模型，请更新 `eval_result_path` 以指向我们的 `estimator_output_path` 变量。

In [ ]:
eval_result_path = keras_output_path
# eval_result_path = estimator_output_path

eval_result = keras_eval_result
# eval_result = estimator_eval_result

### 呈现指标

TFMA 在 [`tfma.experimental.dataframe`](https://tensorflow.google.cn/tfx/model_analysis/api_docs/python/tfma/experimental) 中提供数据帧 API，以将具体化输出加载为 [`Pandas DataFrames`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html)。要查看指标，可以使用`metrics_as_dataframes(tfma.load_metrics(eval_path))`，它会返回一个可能包含多个 DataFrame 的对象，每个 DataFrame 对应一种指标值类型（`double_value`、`confusion_matrix_at_thresholds`、`bytes_value` 和 `array_value`）。填充的具体 DataFrame 取决于评估结果。在这里，我们以 `double_value` DataFrame 为例。

In [ ]:
import tensorflow_model_analysis.experimental.dataframe as tfma_dataframe
dfs = tfma_dataframe.metrics_as_dataframes(
  tfma.load_metrics(eval_result_path))

display(dfs.double_value.head())

每个 DataFrame 都有一个列多重索引，其中包含顶级列：`slices`、`metric_keys` 和 `metric_values`。每组的确切列可以根据有效负载而变化。我们可以使用 `DataFrame.columns` API 检查所有多索引列。例如，切片列为 'Overall'、'trip_start_day'、'trip_start_hour' 和 'trip_start_month'，由 `eval_config` 中的 `slicing_specs` 配置。

In [ ]:
print(dfs.double_value.columns)

### 自动回转

DataFrame 的设计十分详细，因此有效载荷中的信息不会丢失。不过，有时为了直接使用，我们可能希望以更简洁但有损失的形式组织信息：切片作为行，指标作为列。为此，TFMA 提供了 `auto_pivot` API。该实用工具以 `metric_keys` 内的所有非唯一列为中心，并默认将所有切片压缩为一个 `stringified_slices` 列。

In [ ]:
tfma_dataframe.auto_pivot(dfs.double_value).head()

### 筛选切片

由于输出是 DataFrame，任何原生 DataFrame API 都可用于对 DataFrame 进行切片。例如，如果我们只对 1、3、5、7 的 `trip_start_hour` 感兴趣，而对 `trip_start_day` 不感兴趣，则可以使用 DataFrame 的 `.loc` 来筛选逻辑。执行筛选后，我们再次使用 `auto_pivot` 函数在切片与指标视图中重新组织 DataFrame。

In [ ]:
df_double = dfs.double_value
df_filtered = (df_double
  .loc[df_double.slices.trip_start_hour.isin([1,3,5,7])]
)
display(tfma_dataframe.auto_pivot(df_filtered))

### 按指标值排序

我们还可以按指标值对切片进行排序。作为示例，我们展示了如何通过 AUC 升序对上述 DataFrame 中的切片进行排序，以便可以找到性能较差的切片。这涉及两个步骤：一是自动回转以便将切片表示为行，将指标表示为列，二是按 AUC 列对回转后的 DataFrame 进行排序。

In [ ]:
# Pivoted table sorted by AUC in ascending order.
df_sorted = (
    tfma_dataframe.auto_pivot(df_double)
    .sort_values(by='auc', ascending=True)
    )
display(df_sorted.head())

### 呈现绘图

可以使用 [`tfma.view.render_plot`](https://tensorflow.google.cn/tfx/model_analysis/api_docs/python/tfma/view/render_plot) 显示作为训练后 `metric_specs` 添加到 `tfma.EvalConfig` 的任何绘图。

与指标相同的是，可以按切片查看绘图。与指标不同的是，只能显示特定切片值的绘图。因此，必须使用 `tfma.SlicingSpec`，并且必须同时指定切片的特征名称和值。如果未提供切片，则使用 `Overall` 切片的绘图。

在下面的示例中，我们显示的是为 `trip_start_hour:1` 切片计算的 `CalibrationPlot` 和 `ConfusionMatrixPlot` 绘图。

In [ ]:
tfma.view.render_plot(
    eval_result,
    tfma.SlicingSpec(feature_values={'trip_start_hour': '1'}))

## 持续跟踪模型性能

您的训练数据集将用于训练您的模型，并且希望能够代表测试数据集以及将在生产环境中发送到模型的数据。但是，尽管推断请求中的数据可能与训练数据相同，但在许多情况下，数据会开始发生变化，足以使模型的性能随之变化。

这意味着您需要持续监控并衡量模型的性能，以便及时意识到变化并对变化做出反应。我们来看看 TFMA 能提供哪些帮助。

我们来加载 3 种不同的模型，并使用 TFMA 查看它们如何使用 [`render_time_series`](https://tensorflow.google.cn/tfx/model_analysis/api_docs/python/tfma/view/render_time_series) 进行比较。

In [ ]:
# Note this re-uses the EvalConfig from the keras setup.

# Run eval on each saved model
output_paths = []
for i in range(3):
  # Create a tfma.EvalSharedModel that points at our saved model.
  eval_shared_model = tfma.default_eval_shared_model(
      eval_saved_model_path=os.path.join(MODELS_DIR, 'keras', str(i)),
      eval_config=keras_eval_config)

  output_path = os.path.join(OUTPUT_DIR, 'time_series', str(i))
  output_paths.append(output_path)

  # Run TFMA
  tfma.run_model_analysis(eval_shared_model=eval_shared_model,
                          eval_config=keras_eval_config,
                          data_location=tfrecord_file,
                          output_path=output_path)

首先，假设我们昨天已经训练并部署了模型，现在我们想看看它在今天收到的新数据上的表现。可视化效果将从显示 AUC 开始。您可以通过界面进行以下操作：

- 使用“Add metric series”菜单添加其他指标
- 点击“x”关闭不需要的计算图
- 将鼠标悬停在数据点（计算图中线段的末端）上可查看详情

注：在指标序列图表中，X 轴是正在检查的模型运行的模型目录名称。这些名称本身没有意义。

In [ ]:
eval_results_from_disk = tfma.load_eval_results(output_paths[:2])

tfma.view.render_time_series(eval_results_from_disk)

现在，我们假设又过去了一天，并且想看看与前两天相比，它在今天收到的新数据上的表现。

In [ ]:
eval_results_from_disk = tfma.load_eval_results(output_paths)

tfma.view.render_time_series(eval_results_from_disk)

## 模型验证

可以将 TFMA 配置为同时评估多个模型。这样做通常是为了将新模型与基准（例如当前的应用模型）进行比较，以确定指标（例如 AUC 等）相对于基准的性能差异。配置[阈值](https://tensorflow.google.cn/tfx/model_analysis/api_docs/python/tfma/MetricThreshold)时，TFMA 将生成一个 [`tfma.ValidationResult`](https://tensorflow.google.cn/tfx/model_analysis/api_docs/python/tfma/ValidationResult) 记录，指示性能是否符合预期。

我们来重新配置一下 Keras 评估以比较两个模型：候选模型和基准模型。我们还将通过在 AUC 指标上设置 [`tmfa.MetricThreshold`](https://tensorflow.google.cn/tfx/model_analysis/api_docs/python/tfma/MetricThreshold) 来验证候选模型的性能是否符合基准。

In [ ]:
# Setup tfma.EvalConfig setting
eval_config_with_thresholds = text_format.Parse("""
  ## Model information
  model_specs {
    name: "candidate"
    # For keras we need to add a `label_key`.
    label_key: "big_tipper"
  }
  model_specs {
    name: "baseline"
    # For keras we need to add a `label_key`.
    label_key: "big_tipper"
    is_baseline: true
  }

  ## Post training metric information
  metrics_specs {
    metrics { class_name: "ExampleCount" }
    metrics { class_name: "BinaryAccuracy" }
    metrics { class_name: "BinaryCrossentropy" }
    metrics {
      class_name: "AUC"
      threshold {
        # Ensure that AUC is always > 0.9
        value_threshold {
          lower_bound { value: 0.9 }
        }
        # Ensure that AUC does not drop by more than a small epsilon
        # e.g. (candidate - baseline) > -1e-10 or candidate > baseline - 1e-10
        change_threshold {
          direction: HIGHER_IS_BETTER
          absolute { value: -1e-10 }
        }
      }
    }
    metrics { class_name: "AUCPrecisionRecall" }
    metrics { class_name: "Precision" }
    metrics { class_name: "Recall" }
    metrics { class_name: "MeanLabel" }
    metrics { class_name: "MeanPrediction" }
    metrics { class_name: "Calibration" }
    metrics { class_name: "CalibrationPlot" }
    metrics { class_name: "ConfusionMatrixPlot" }
    # ... add additional metrics and plots ...
  }

  ## Slicing information
  slicing_specs {}  # overall slice
  slicing_specs {
    feature_keys: ["trip_start_hour"]
  }
  slicing_specs {
    feature_keys: ["trip_start_day"]
  }
  slicing_specs {
    feature_keys: ["trip_start_month"]
  }
  slicing_specs {
    feature_keys: ["trip_start_hour", "trip_start_day"]
  }
""", tfma.EvalConfig())

# Create tfma.EvalSharedModels that point at our keras models.
candidate_model_path = os.path.join(MODELS_DIR, 'keras', '2')
baseline_model_path = os.path.join(MODELS_DIR, 'keras', '1')
eval_shared_models = [
  tfma.default_eval_shared_model(
      model_name=tfma.CANDIDATE_KEY,
      eval_saved_model_path=candidate_model_path,
      eval_config=eval_config_with_thresholds),
  tfma.default_eval_shared_model(
      model_name=tfma.BASELINE_KEY,
      eval_saved_model_path=baseline_model_path,
      eval_config=eval_config_with_thresholds),
]

validation_output_path = os.path.join(OUTPUT_DIR, 'validation')

# Run TFMA
eval_result_with_validation = tfma.run_model_analysis(
    eval_shared_models,
    eval_config=eval_config_with_thresholds,
    data_location=tfrecord_file,
    output_path=validation_output_path)

使用一个或多个模型针对基准运行评估时，TFMA 会自动为评估期间计算的所有指标添加差异指标。这些指标以相应的指标命名，但会在指标名称后附加 `_diff`。

我们来看一下运行生成的指标：

In [ ]:
tfma.view.render_time_series(eval_result_with_validation)

现在，我们来看一下验证检查的输出。要查看验证结果，我们需要使用 [`tfma.load_validator_result`](https://tensorflow.google.cn/tfx/model_analysis/api_docs/python/tfma/load_validation_result)。对于我们的示例，验证失败，因为 AUC 低于阈值。

In [ ]:
validation_result = tfma.load_validation_result(validation_output_path)
print(validation_result.validation_ok)

# Copyright © 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

注：本网站提供的应用所使用的数据来自原始源（www.cityofchicago.org，芝加哥市官方网站），但在使用时进行了修改。芝加哥市不对本网站提供的任何数据的内容、准确性、时效性或完整性承担任何责任。本网站提供的数据可能会随时更改。您了解并同意，使用本网站提供的数据须自担风险。